# Import Necessary Modules

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf

# Import Datasets to Pandas

In [2]:
# Two datasets: training set and test set
pathTrain = r'ChurnModeling.csv'
pathTest = r'ChurnModeling_Test.csv'

## Training Data

In [3]:
dataTrain = pd.read_csv(pathTrain, sep=',')
# make sure sep is set correctly
# add list of name parameters if parsing error due to rows having too many cols

In [4]:
# take a quick look to see if everything is okay
dataTrain.head()
print(dataTrain.shape)

(10000, 14)


In [5]:
# Not sure how to handle all the dummy variables
# pd.get_dummies creates a column for each unique surname. Obviously this will not be helpful
# I propose to drop 'Surname' as well as 'Customer ID' and 'RowNumber' from the dataset
print(pd.get_dummies(dataTrain).shape)

(10000, 2948)


In [6]:
# First I'll split the dataTrain df into regressors X and outcome Y
XdataTrain = dataTrain.drop(columns=['RowNumber', 'Surname', 'Exited'])
y_train = dataTrain['Exited']

In [7]:
# Convert categorical variables to dummy variables
pd.get_dummies(XdataTrain, columns=['Gender', 'Geography'], drop_first=True).head()

,CustomerId,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Gender_Male,Geography_Germany,Geography_Spain
0,15634602,619,42,2,0.00,1,1,1,101348.88,0,0,0
1,15647311,608,41,1,83807.86,1,0,1,112542.58,0,0,1
2,15619304,502,42,8,159660.80,3,1,0,113931.57,0,0,0
3,15701354,699,39,1,0.00,2,0,0,93826.63,0,0,0
4,15737888,850,43,2,125510.82,1,1,1,79084.10,0,0,1


In [8]:
X_train = pd.get_dummies(XdataTrain, columns=['Gender', 'Geography'], drop_first=True)

In [9]:
# check that X_train has the same number of rows as dataTrain
print(X_train.shape)
print(X_train.shape[0] == dataTrain.shape[0])

(10000, 12)
True


## Test Data

In [72]:
dataTest = pd.read_csv(pathTest, sep=',')

In [73]:
print(dataTest.shape)
dataTest.head()

(1000, 14)


,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,10001,15798485,Copley,565,France,Male,31,1,0.00,1,0,1,20443.08,0
1,10002,15588959,T'ang,569,France,Male,34,4,0.00,1,0,1,4045.90,0
2,10003,15624896,Ku,669,France,Female,20,7,0.00,2,1,0,128838.67,0
3,10004,15639629,McConnan,694,France,Male,39,4,173255.48,1,1,1,81293.10,0
4,10005,15638852,Ts'ui,504,Spain,Male,28,10,109291.36,1,1,1,187593.15,0


In [74]:
# Split the dataTest df into regressors X and outcome y
X_test = dataTest.drop(columns=['RowNumber', 'Surname', 'Exited'])
y_test = dataTest['Exited']

In [75]:
X_test.head()

,CustomerId,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,15798485,565,France,Male,31,1,0.00,1,0,1,20443.08
1,15588959,569,France,Male,34,4,0.00,1,0,1,4045.90
2,15624896,669,France,Female,20,7,0.00,2,1,0,128838.67
3,15639629,694,France,Male,39,4,173255.48,1,1,1,81293.10
4,15638852,504,Spain,Male,28,10,109291.36,1,1,1,187593.15


In [76]:
# Convert categorical variables into dummy variables in X_test
X_test = pd.get_dummies(X_test, columns=['Gender', 'Geography'], drop_first=True)

In [77]:
# check that X_test has the same number of rows as dataTrain
print(X_test.shape)
print(X_test.shape[0] == dataTest.shape[0])

(1000, 12)
True


## ========================================

In [16]:
print(X_train.head())
print(y_train.head())
print(X_test.head())
print(y_test.head())

   CustomerId  CreditScore  Age  Tenure    Balance  NumOfProducts  HasCrCard  \
0    15634602          619   42       2       0.00              1          1   
1    15647311          608   41       1   83807.86              1          0   
2    15619304          502   42       8  159660.80              3          1   
3    15701354          699   39       1       0.00              2          0   
4    15737888          850   43       2  125510.82              1          1   

   IsActiveMember  EstimatedSalary  Gender_Male  Geography_Germany  \
0               1        101348.88            0                  0   
1               1        112542.58            0                  0   
2               0        113931.57            0                  0   
3               0         93826.63            0                  0   
4               1         79084.10            0                  0   

   Geography_Spain  
0                0  
1                1  
2                0  
3             

So far so good

## ========================================

# Using The Training Data to Make a Model

using a backwards elimination approach  

#### Trial I

In [17]:
X_train.head()

,CustomerId,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Gender_Male,Geography_Germany,Geography_Spain
0,15634602,619,42,2,0.00,1,1,1,101348.88,0,0,0
1,15647311,608,41,1,83807.86,1,0,1,112542.58,0,0,1
2,15619304,502,42,8,159660.80,3,1,0,113931.57,0,0,0
3,15701354,699,39,1,0.00,2,0,0,93826.63,0,0,0
4,15737888,850,43,2,125510.82,1,1,1,79084.10,0,0,1


In [18]:
logRegr = smf.Logit(y_train, X_train.drop(columns=['CustomerId'])).fit()

Optimization terminated successfully.
         Current function value: 0.438017
         Iterations 6


In [19]:
logRegr.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                 Exited   No. Observations:                10000
Model:                          Logit   Df Residuals:                     9989
Method:                           MLE   Df Model:                           10
Date:                Thu, 14 Feb 2019   Pseudo R-squ.:                  0.1335
Time:                        10:12:09   Log-Likelihood:                -4380.2
converged:                       True   LL-Null:                       -5054.9
                                        LLR p-value:                8.128e-284
=====================================================================================
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
CreditScore          -0.0036      0.000    -18.882      0.000      -0.004      -0.003
Age                   0.0586      0.002     25.041      0.000       0.054       0.063
Tenure               -0.0399      0.009     -4.396      0.000      -0.058      -0.022
Balance             9.64e-07   4.95e-07      1.948      0.051   -6.03e-09    1.93e-06
NumOfProducts        -0.3309      0.045     -7.342      0.000      -0.419      -0.243
HasCrCard            -0.1938      0.057     -3.408      0.001      -0.305      -0.082
IsActiveMember       -1.0736      0.056    -19.010      0.000      -1.184      -0.963
EstimatedSalary   -8.486e-07   4.57e-07     -1.856      0.064   -1.74e-06    4.77e-08
Gender_Male          -0.6193      0.053    -11.616      0.000      -0.724      -0.515
Geography_Germany     0.7887      0.067     11.720      0.000       0.657       0.921
Geography_Spain      -0.0415      0.069     -0.597      0.550      -0.178       0.095
=====================================================================================
"""

Spain has p value of .550. I will exclude it

#### Trial II

In [20]:
logRegr = smf.Logit(y_train, X_train.drop(columns=['CustomerId', 'Geography_Spain'])).fit()

Optimization terminated successfully.
         Current function value: 0.438035
         Iterations 6


In [21]:
logRegr.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                 Exited   No. Observations:                10000
Model:                          Logit   Df Residuals:                     9990
Method:                           MLE   Df Model:                            9
Date:                Thu, 14 Feb 2019   Pseudo R-squ.:                  0.1334
Time:                        10:12:09   Log-Likelihood:                -4380.3
converged:                       True   LL-Null:                       -5054.9
                                        LLR p-value:                7.709e-285
=====================================================================================
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
CreditScore          -0.0036      0.000    -19.039      0.000      -0.004      -0.003
Age                   0.0586      0.002     25.042      0.000       0.054       0.063
Tenure               -0.0399      0.009     -4.401      0.000      -0.058      -0.022
Balance            9.581e-07   4.95e-07      1.936      0.053   -1.17e-08    1.93e-06
NumOfProducts        -0.3320      0.045     -7.371      0.000      -0.420      -0.244
HasCrCard            -0.1941      0.057     -3.413      0.001      -0.306      -0.083
IsActiveMember       -1.0740      0.056    -19.019      0.000      -1.185      -0.963
EstimatedSalary   -8.535e-07   4.57e-07     -1.867      0.062   -1.75e-06    4.26e-08
Gender_Male          -0.6199      0.053    -11.630      0.000      -0.724      -0.515
Geography_Germany     0.8026      0.063     12.695      0.000       0.679       0.926
=====================================================================================
"""

#### Trial III

In [22]:
logRegr = smf.Logit(y_train, X_train.drop(columns=['CustomerId', 'Geography_Spain', 'EstimatedSalary'])).fit()

Optimization terminated successfully.
         Current function value: 0.438209
         Iterations 6


In [23]:
logRegr.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                 Exited   No. Observations:                10000
Model:                          Logit   Df Residuals:                     9991
Method:                           MLE   Df Model:                            8
Date:                Thu, 14 Feb 2019   Pseudo R-squ.:                  0.1331
Time:                        10:12:09   Log-Likelihood:                -4382.1
converged:                       True   LL-Null:                       -5054.9
                                        LLR p-value:                3.263e-285
=====================================================================================
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
CreditScore          -0.0037      0.000    -19.845      0.000      -0.004      -0.003
Age                   0.0582      0.002     24.977      0.000       0.054       0.063
Tenure               -0.0408      0.009     -4.506      0.000      -0.059      -0.023
Balance            8.996e-07   4.94e-07      1.822      0.068   -6.81e-08    1.87e-06
NumOfProducts        -0.3397      0.045     -7.570      0.000      -0.428      -0.252
HasCrCard            -0.1977      0.057     -3.481      0.000      -0.309      -0.086
IsActiveMember       -1.0727      0.056    -19.009      0.000      -1.183      -0.962
Gender_Male          -0.6212      0.053    -11.657      0.000      -0.726      -0.517
Geography_Germany     0.8032      0.063     12.704      0.000       0.679       0.927
=====================================================================================
"""

In [24]:
from math import log10

In [31]:
X_train['log_Balance'] = X_train['Balance'].transform(lambda x: log10(x+1))

In [32]:
logRegr_fin = smf.Logit(y_train, X_train.drop(columns=['CustomerId', 'Geography_Spain', 'Balance', 'HasCrCard'])).fit()

Optimization terminated successfully.
         Current function value: 0.438697
         Iterations 6


In [33]:
logRegr_fin.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                 Exited   No. Observations:                10000
Model:                          Logit   Df Residuals:                     9991
Method:                           MLE   Df Model:                            8
Date:                Thu, 14 Feb 2019   Pseudo R-squ.:                  0.1321
Time:                        10:16:45   Log-Likelihood:                -4387.0
converged:                       True   LL-Null:                       -5054.9
                                        LLR p-value:                4.199e-283
=====================================================================================
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
CreditScore          -0.0037      0.000    -19.674      0.000      -0.004      -0.003
Age                   0.0580      0.002     24.882      0.000       0.053       0.063
Tenure               -0.0415      0.009     -4.583      0.000      -0.059      -0.024
NumOfProducts        -0.3461      0.045     -7.649      0.000      -0.435      -0.257
IsActiveMember       -1.0706      0.056    -18.989      0.000      -1.181      -0.960
EstimatedSalary   -8.943e-07   4.57e-07     -1.957      0.050   -1.79e-06    1.15e-09
Gender_Male          -0.6224      0.053    -11.694      0.000      -0.727      -0.518
Geography_Germany     0.8095      0.065     12.476      0.000       0.682       0.937
log_Balance           0.0175      0.013      1.326      0.185      -0.008       0.043
=====================================================================================
"""

In [37]:
pd.DataFrame(data=[y_train, logRegr_fin.predict()], columns=['Result', 'Prediction'])

,Result,Prediction
Exited,NaN,NaN
Unnamed 0,NaN,NaN


In [39]:
type(y_train)

pandas.core.series.Series

In [52]:
forecast = pd.DataFrame(logRegr_fin.predict(), columns=['yHat']).join(pd.DataFrame(y_train))

In [54]:
forecast.sort_values(by='yHat', ascending=False)

,yHat,Exited
3531,0.931474,1
2473,0.904638,1
7499,0.888859,1
416,0.888647,1
8156,0.876845,0
2115,0.875978,1
4501,0.872579,0
8188,0.871918,0
4815,0.867486,0
5004,0.863306,1


In [55]:
logRegr_fin.pred_table()

array([[7696.,  267.],
       [1605.,  432.]])

In [58]:
logRegr_fin.summary2()

<class 'statsmodels.iolib.summary2.Summary'>
"""
                          Results: Logit
==================================================================
Model:              Logit            Pseudo R-squared: 0.132      
Dependent Variable: Exited           AIC:              8791.9385  
Date:               2019-02-14 11:28 BIC:              8856.8316  
No. Observations:   10000            Log-Likelihood:   -4387.0    
Df Model:           8                LL-Null:          -5054.9    
Df Residuals:       9991             LLR p-value:      4.1989e-283
Converged:          1.0000           Scale:            1.0000     
No. Iterations:     6.0000                                        
------------------------------------------------------------------
                   Coef.  Std.Err.    z     P>|z|   [0.025  0.975]
------------------------------------------------------------------
CreditScore       -0.0037   0.0002 -19.6741 0.0000 -0.0041 -0.0033
Age                0.0580   0.0023  24.8818 0.0000  0.0535  0.0626
Tenure            -0.0415   0.0091  -4.5835 0.0000 -0.0593 -0.0238
NumOfProducts     -0.3461   0.0452  -7.6494 0.0000 -0.4348 -0.2574
IsActiveMember    -1.0706   0.0564 -18.9888 0.0000 -1.1811 -0.9601
EstimatedSalary   -0.0000   0.0000  -1.9574 0.0503 -0.0000  0.0000
Gender_Male       -0.6224   0.0532 -11.6937 0.0000 -0.7268 -0.5181
Geography_Germany  0.8095   0.0649  12.4759 0.0000  0.6824  0.9367
log_Balance        0.0175   0.0132   1.3264 0.1847 -0.0084  0.0434
==================================================================

"""

# Running Test Data Through Model

In [79]:
# working with X_test and y_test

# need to modify X_test in the same way I modified X_train

X_test['log_balance'] = X_test['Balance'].transform(lambda x: log10(x+1))

In [81]:
# Run test!
testPred = logRegr_fin.predict(X_test.drop(columns=['CustomerId', 'Geography_Spain', 'Balance', 'HasCrCard']))

In [82]:
testResults = pd.DataFrame(testPred, columns=['yHat']).join(pd.DataFrame(y_test))

In [85]:
testResults.head(10)

,yHat,Exited
0,0.084301,0
1,0.088203,0
2,0.082652,0
3,0.076963,0
4,0.059020,0
5,0.021630,0
6,0.048214,0
7,0.075827,0
8,0.163866,0
9,0.089016,0


In [87]:
testResults.sort_values(by='yHat', ascending=False)

,yHat,Exited
957,0.900729,1
997,0.862914,0
693,0.851773,0
850,0.806834,1
249,0.801774,1
260,0.801656,1
892,0.788823,1
834,0.761197,0
944,0.761119,1
740,0.731776,1
